Lets go through the steps that the robot takes during play.

Turn order:
(1) Determine the order of play - the robot doesn't need to know this until we start coding some AI decision making.

(2) Auction power plants
    Robot needs to know
     - what power plants are available
     - what power plants it has
     - what power plants other players have chosen for bidding
     - how many cities it has
    Robot will decide
     - if it needs to buy a new power plant or not
     - which power plant it will choose for bidding on
     - if it will join in the public bidding process
     - what the maximum it will pay for the power plant is
     - which power plant to discard if necessary
    Robot will update
     - what power plants it has
        - how many spaces it has for resources
     - how its resources are distributed
        - do any resources need throwing away
     - how much money it has

(3) Buying resources
    Robot needs to know
     - what space it has for resources
     - what resources it has
     - the cost of available resources
    Robot will decide
     - what resources it will buy
    Robot will update
     - what resources it has
     - where those resources are stored
     - how much money it has

(4) Building
    Robot needs to know
     - what the map is
        - what cities there are
        - what the connection costs are
     - what step we are in (step 1, 2 or 3)
     - what cities it has built on
     - what cities other players have built on
        - how many people have built there
        - what spaces are free, and their cost
     Robot will decide
      - if it will build a city
      - where to build the city
      - if it has enough money to build another city
     Robot will update
      - how much money it has
      - how many cities it has
        - where those cities are

(5) Bureaucracy
    Robot needs to know
     - how many cities it has built on
     - how many cities its power plants can supply
     - what resources it has
    Robot will decide
     - which resources to use to power the most cities
    The robot will then update
     - how much money it has
     - what resources it has
    


Variables and databases I need:
- a representation of the map
    - the cities on the map
    - the connections between the cities
    - the regions on the map
- the resource market
- the power plants
- a list of money earned depending on cities powered

- money
- each type of resource
- a list of power plants owned
- a list of cities in the robot's network

In [1]:
# What libraries / functions do we need to import? Put them in here.

import pyodbc
from pyodbc import Error

from databaseConnection import create_db_connection, read_query

from powerPlants import PowerPlant, createMarket

from resourceMarket import resourceMarket

# import pwinput as pin

In [2]:
# Set the starting variables

robotMoney = 50
mapName = "Japan"
powerPlantDeck = "standard"
step = 1

# Security note - public versions of this will not be asking for log in details to a database, as
# the people using it wouldn't have those credentials. This is only for development purposes while
# I play with the code.
# Database access details:
user_name = input("Please give the user name for the database access.")
user_password = input(f"Please give the password for {user_name}")

# user_password = pin.pwinput(prompt = f"Please give the password for {user_name}", mask = '*')
# This doesn't work with jupyter notebooks - the prompt appears but the cursor moves on to the next
# cell and it is not possible to enter in the password. I have not found anything about this issue
# on stackoverflow or by Googling - which surprises me.

# We will also need to set the robot options, but we'll code those later on

Please give the user name for the database access.Chris
Please give the password for ChrisT3sting0ut@H0me


In [3]:
# Actually, we should initialise the instance of the Robot, so that it has money for the auction phase.

from robotClass import Robot

robot = Robot(robotMoney)

Robot.setRobotName(robot)

print(robot)


What should we call this robot?Harriet
Harriet has: Money = 50 and these powerplants: []


In [ ]:
# Phase 2: Power plant auction - create the power plant market

# We ask the user to tell us the costs of the current power plants in the market:
listOfPowerPlants = createMarket(step)

In [ ]:

power_plants = PowerPlant.updatePowerPlantMarket(user_name, user_password, listOfPowerPlants)

# We print some details to see what we have got.
print(listOfPowerPlants)
print(power_plants[1])


In [ ]:
# Now we will bid on some power plants
# We will start with the robot rule 'second smallest number'

# If I have a mix of 'print' and 'input' messages, the printed messages appear below this block, while the input
# messages appear in a box. This is confusing and hides some of the information I am trying to communicate.
# BUT when I view the Jypter notebook as a webpage, that problem goes away....

# The robot will ask if it is their turn.
myTurn = input("Is it my turn to choose a power plant to bid on? Y or N?")

didIbuyYet = "n"

# If it is step 1 or 2, we need to split the existing market into the current and future market.
# If it is step 3, then all 6 power plants are in the current market
# Only the current market is of interest to the robot.

def createCurrentMarket(listOfPowerPlants):    
    if step == 3:
        currentMarket = listOfPowerPlants
    else:
        listOfPowerPlants.sort()
        currentMarket = listOfPowerPlants[:-4]
        
    return currentMarket

currentMarket = createCurrentMarket(listOfPowerPlants)
print(f"The current market is {currentMarket}")

# I should put in a comment as to why I am doing this here.
# What am I doing here????
# I don't think I need this - but I will just comment it out.
# currentMarketDetails = PowerPlant.updatePowerPlantMarket(user_name, user_password, currentMarket)

# If it is not the robot's turn to choose, then for this robot we just need to update
# the market. For other versions, we will need to check what power plant is being bid on
# and what the bids are - but that is for later on.

while didIbuyYet == 'n':

    while myTurn.lower() == "n":
        boughtPowerPlant = input("I won't bid, so just tell me which power plant was bought. ")
        listOfPowerPlants.remove(int(boughtPowerPlant))
        # This code is repeated in the block below, so we could hive it off as a function
        # when we refactor.
        newPowerPlant = int(input("What is the cost of the new power plant you have drawn? "))
        listOfPowerPlants.append(newPowerPlant)
        currentMarket = createCurrentMarket(listOfPowerPlants)
        print(f"Power plant {newPowerPlant} was drawn and the current market is {currentMarket}")
        myTurn = input("Is it now my turn to choose a power plant to bid on? Y or N? ")
    
    # If it is the robot's turn to choose, then they will keep on choosing until they buy something.
    # So we only care if 'myTurn' is 'n' - otherwise we just run this block.
    choice = currentMarket[1]
    print(f"I bid {choice} electros for power plant {choice}.")
    otherBid = input(f"Does anyone else want to bid on power plant {choice}? Y or N? ")
    if otherBid.lower() == "y":
        print(f"Ok, I pass on {choice}.")
        listOfPowerPlants.remove(choice)
        # The next 4 lines are a repeat of lines above, so we can hive it off as a function later.
        newPowerPlant = int(input("What is the cost of the new power plant you have drawn? "))
        listOfPowerPlants.append(newPowerPlant)
        currentMarket = createCurrentMarket(listOfPowerPlants)
        print(f"Power plant {newPowerPlant} was drawn and the current market is {currentMarket}")
    elif otherBid.lower() == "n":
        didIbuyYet = 'y'
        print(f"Ok, I will buy power plant {choice} for {choice} electros")

    


Robot.deductRobotMoney(robot, choice)
print(f"I have spent {choice} electros, and now I have {robot.money} electros.")
Robot.addPowerplant(robot, choice)
print(f"I now have these power plants: {robot.powerPlants}")
    
    



In [4]:
# For testing purposes, let's populate the robot's powerplant list without doing the above tasks.
robot.powerPlants = []
robot.money = 100

examplePowerPlants = (3, 33, 28)

for choice in examplePowerPlants:
    Robot.deductRobotMoney(robot, choice)
    print(f"I have spent {choice} electros, and now I have {robot.money} electros.")
    Robot.addPowerplant(robot, choice)
    print(f"I now have these power plants: {robot.powerPlants}")


I have spent 3 electros, and now I have 97 electros.
I now have these power plants: [3]
I have spent 33 electros, and now I have 64 electros.
I now have these power plants: [3, 33]
I have spent 28 electros, and now I have 36 electros.
I now have these power plants: [3, 33, 28]


In [5]:
# Now that the robot has a power plant, it needs to decide what resources to buy.
# For now it will just buy basic resources to power the power plant for 1 turn.
# It needs to know:
# What is the price point of all the resources
# What resources it already has

# if there are x resources of type y, then the price of the next resource is a.
# if I organise it that way, then each time the robot buys 1 resource, we need to check
# the price of the next piece of resource.
# I might change that later - so let's make a separate function that is called
# to get the price of the resource.

# Deciding what the robot needs to buy:
# We will start with a robot that only buys the basic requirements for 
# the power plants they have - so they need to know:
# what resources do their power plants need?
# what resources do they have at the moment?

# function - get info about the resources their power plants need

currentRobotPowerPlants = PowerPlant.updateRobotPowerPlants(user_name, user_password, robot.powerPlants)

for x in currentRobotPowerPlants:
    print(str(x))

# Let's work out how much of each resource we want to buy
# Identify the type of resource and how many are needed.
# create a shopping basket that has each resource type and 0 for each value

shoppingBasket = {'coal': 0, 'oil': 0, 'garbage': 0, 'uranium': 0}

for x in currentRobotPowerPlants:
    try:
        shoppingBasket[getattr(x, 'resource')] += getattr(x, 'powerCost')
    except KeyError:
        # Green power plants do not have a 'resource', so they return an error
        print(f"Power plant {getattr(x, 'cost')} does not need any resources.")
        
print(f"The shopping basket has: {shoppingBasket}")


MySQL Database connection successful
Cost: 3, Type: Coal, Resource: coal, Resource2: None, PowerCost: 1, CitiesPowered: 1
Cost: 33, Type: Garbage, Resource: garbage, Resource2: None, PowerCost: 2, CitiesPowered: 5
Cost: 28, Type: Coal, Resource: coal, Resource2: None, PowerCost: 3, CitiesPowered: 5
The shopping basket has: {'coal': 4, 'oil': 0, 'garbage': 2, 'uranium': 0}


In [18]:
# The Japan map starts with coal 2 Elektro, oil 4 Elektro, garbage 6 Elektro and uranium 12 Elektro
# So coal = 21; oil = 16; garbage = 9; uranium =3

currentResourceMarket = resourceMarket(coal = 21, oil = 16, garbage = 9, uranium = 3)

print(currentResourceMarket)

# it will need to have a method for determining the cost of the next resource
# or perhaps a way to calculate how much buying x pieces of a resource would cost
# (because that will help the robot in deciding what to buy)

# Because we are not tracking what resources the other players are buying, we should 
# ask the player to tell us the current state of the resourceMarket at the 
# point when the robot needs to buy resources. We will ask what the current price is, and 
# how many resources in that bracket - to avoid the player having to count everything.

connection = create_db_connection("localhost", user_name, user_password, "robots")

japanResourceMarket = {}
resources = ['coal', 'oil', 'garbage', 'uranium']

for x in resources:
    query = f"SELECT * FROM {x}"
    result = read_query(connection, query)
    japanResourceMarket[x] = dict(result)

print(japanResourceMarket)


The current resource market is: Coal 21; Oil 16; Garbage 9; Uranium 3.
MySQL Database connection successful
{'coal': {1: 8, 2: 8, 3: 8, 4: 7, 5: 7, 6: 7, 7: 6, 8: 6, 9: 6, 10: 5, 11: 5, 12: 5, 13: 4, 14: 4, 15: 4, 16: 3, 17: 3, 18: 3, 19: 2, 20: 2, 21: 2, 22: 1, 23: 1, 24: 1}, 'oil': {1: 8, 2: 8, 3: 8, 4: 7, 5: 7, 6: 7, 7: 6, 8: 6, 9: 6, 10: 5, 11: 5, 12: 5, 13: 4, 14: 4, 15: 4, 16: 3, 17: 3, 18: 3, 19: 2, 20: 2, 21: 2, 22: 1, 23: 1, 24: 1}, 'garbage': {1: 8, 2: 8, 3: 8, 4: 7, 5: 7, 6: 7, 7: 6, 8: 6, 9: 6, 10: 5, 11: 5, 12: 5, 13: 4, 14: 4, 15: 4, 16: 3, 17: 3, 18: 3, 19: 2, 20: 2, 21: 2, 22: 1, 23: 1, 24: 1}, 'uranium': {1: 16, 2: 14, 3: 12, 4: 10, 5: 8, 6: 7, 7: 6, 8: 5, 9: 4, 10: 3, 11: 2, 12: 1}}


In [8]:
# How much will 3 pieces of coal cost me?

def multipleResourceCost(resource, number, currentResourceMarket):
    try:
        placeMarker = getattr(currentResourceMarket, resource)
    except TypeError:
        # This occurs when this function tries to get the resources that
        # a green powerplant needs. As it doesn't need any resources, we
        # will return 0
        print("Green power plants don't use resources.")
        return 0
    cost = 0
    while number > 0:
        cost += japanResourceMarket[resource][placeMarker]
        placeMarker -= 1
        number -= 1
    return cost

def totalBasketCost(shoppingBasket, currentResourceMarket):
    # I have specified the 4 resources because the basket may also include wind
    # which I don't want to try to check on the resource market for.
    resources = ['coal', 'oil', 'garbage', 'uranium']
    totalCost = 0
    for resource in resources:
        number = shoppingBasket[resource]
        totalCost += multipleResourceCost(resource, number, currentResourceMarket)
    return totalCost

totalShoppingBasketCost = totalBasketCost(shoppingBasket, currentResourceMarket)

print(f"My shopping basket contains {shoppingBasket}.")
print(f"My shopping basket will cost {totalShoppingBasketCost} in total")
    

My shopping basket contains {'coal': 4, 'oil': 0, 'garbage': 2, 'uranium': 0}.
My shopping basket will cost 21 in total


In [ ]:
# Now we have to decide if we will buy the resources or not. Let's start with 'yes'

if totalShoppingBasketCost < robot.money:
    Robot.deductRobotMoney(robot, totalShoppingBasketCost)
    Robot.addResources(robot, shoppingBasket)
# else:
    # Now we need to decide what resources to take out of the basket.
    # First we decide which resources will power the most cities.
    # Note - this will become more complicated when we have resources left over
    # from previous rounds....
    # This is a sorting algorithm
    
    
    
# Because we are only updating the resource market just before the robot
# buys resources, we are not going to try to track the state of the resource
# market. The human players can do that.
print(f"I have bought these resources: {shoppingBasket}. Please remove them from the resource market.")

# We are not going to tell the human players how to distribute those resources on the power plant cards.
# The human players can do that themselves.
print(f"My resources are now {robot.resources}")
print(f"My money is now {robot.money}")

In [19]:
testMoney = 19

def workOutResourceCost(currentRobotPowerPlants):
    # we want to work out the cost of each power plant
    resourceCostList = []
    for x in currentRobotPowerPlants:
        resourceCost = multipleResourceCost(getattr(x, 'resource'), getattr(x, 'powerCost'), currentResourceMarket)
        resourceCostList.append({'cost': getattr(x, 'cost'), 'powerCost': getattr(x, 'powerCost'), 'resourceCost': resourceCost, 'cities': getattr(x, 'citiesPowered'), 'resource': getattr(x, 'resource')}) 
    return resourceCostList


def removeGreenPlants(resourceCostList):
    chosenPowerPlants = []
    plants_to_remove = []

    # remove green power plants with a resource cost of 0 so that we don't divide by 0
    for i, plant in enumerate(resourceCostList):
         if plant['resourceCost'] == 0:
            chosenPowerPlants.append(plant)
            plants_to_remove.append(i)

    for index in reversed(plants_to_remove):
        resourceCostList.pop(index)
    return chosenPowerPlants, resourceCostList
        

def sortPlantsByResourceCost(resourceCostList):
    sortedResourceCostList = sorted(resourceCostList, key=lambda plant: plant['cities'] / plant['resourceCost'], reverse=True)
    return sortedResourceCostList

# TODO: amend 'testMoney' in this function to call the money the robot currently has
def checkIfCanBuyResources(amendedResourceCostList, chosenPowerPlants, testMoney, currentResourceMarket):    
    plant = amendedResourceCostList[0]
    # add this power plant to our chosen list if we can afford the resources
    if plant['resourceCost'] <= testMoney:
        chosenPowerPlants.append(plant)
        testMoney -= plant['resourceCost']
        # We update the currentResourceMarket with a class method
        # that uses the resource key word to adjust the right resource.
        # 'powerCost' = the amount of that resource the power plant spends
        resourceMarket.deductResource(currentResourceMarket, plant['powerCost'], plant['resource'])
    # remove this power plant from the list we are checking
    amendedResourceCostList.pop(0)
    return amendedResourceCostList, chosenPowerPlants, testMoney, currentResourceMarket


### The steps for the robot to decide what resources to buy:

### STEP 1: get a list with resource costs for the power plants.

resourceCostList = workOutResourceCost(currentRobotPowerPlants)
        
print(f"The cost of each power plant is: {resourceCostList}")      

### STEP 2: remove any green power plants from that list.

chosenPowerPlants, amendedResourceCostList = removeGreenPlants(resourceCostList)

### STEP 3: The while loop

while len(amendedResourceCostList) > 0:
    # We start by amending the resource costs for the remaining power plants - this doesn't matter the first time
    # but it is a short list and won't take much time.
    for plant in amendedResourceCostList: 
        plant['resourceCost'] = multipleResourceCost(plant['resource'], plant['powerCost'], currentResourceMarket)
    
    # Sort the remaining power plants by cost to buy the resources they need
    amendedResourceCostList = sortPlantsByResourceCost(amendedResourceCostList)
    
    # check the first power plant on the list, and remove it
    # we also amend the currentResourceMarket and the testMoney levels
    amendedResourceCostList, chosenPowerPlants, testMoney, currentResourceMarket = checkIfCanBuyResources(amendedResourceCostList, chosenPowerPlants, testMoney, currentResourceMarket)
    
    print(f"\nThe cost of each remaining power plant is: {amendedResourceCostList}") 
    print(f"The chosen power plants are: {chosenPowerPlants}")
    print(f"The current resource market is: {currentResourceMarket}")
    print(f"The remaining money is: {testMoney}")


print(f"\nThe sorted list is: {amendedResourceCostList}")
print(f"The chosen power plants are: {chosenPowerPlants}")
print(f"The current resource market is: {currentResourceMarket}")
print(f"The remaining money is: {testMoney}")



The cost of each power plant is: [{'cost': 3, 'powerCost': 1, 'resourceCost': 2, 'cities': 1, 'resource': 'coal'}, {'cost': 33, 'powerCost': 2, 'resourceCost': 12, 'cities': 5, 'resource': 'garbage'}, {'cost': 28, 'powerCost': 3, 'resourceCost': 6, 'cities': 5, 'resource': 'coal'}]

The cost of each remaining power plant is: [{'cost': 3, 'powerCost': 1, 'resourceCost': 2, 'cities': 1, 'resource': 'coal'}, {'cost': 33, 'powerCost': 2, 'resourceCost': 12, 'cities': 5, 'resource': 'garbage'}]
The chosen power plants are: [{'cost': 28, 'powerCost': 3, 'resourceCost': 6, 'cities': 5, 'resource': 'coal'}]
The current resource market is: The current resource market is: Coal 18; Oil 16; Garbage 9; Uranium 3.
The remaining money is: 13

The cost of each remaining power plant is: [{'cost': 3, 'powerCost': 1, 'resourceCost': 3, 'cities': 1, 'resource': 'coal'}]
The chosen power plants are: [{'cost': 28, 'powerCost': 3, 'resourceCost': 6, 'cities': 5, 'resource': 'coal'}, {'cost': 33, 'powerCost':